# 0. Setup

In [ ]:
# Basic imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import librosa
import os
import IPython.display as ipd
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [ ]:
# Torch imports
import torch
import torchaudio
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset, random_split
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchaudio.transforms import MelSpectrogram, Resample
from IPython.display import Audio
import torch.optim as optim

In [ ]:
# Directories
DATA_DIR = "data/"
AUDIO_DIR = DATA_DIR + "train_audio/"

# Preprocessing info
SAMPLE_RATE = 32000 # All our audio uses this sample rate
SAMPLE_LENGTH = 5 # Duration we want to crop our audio to
NUM_SPECIES = 182 # Number of bird species we need to label

# Training hyperparameters
BATCH_SIZE = 128 # Number of samples per batch while training our network
NUM_EPOCHS = 3 # Number of epochs to train our network
LEARNING_RATE = 0.001 # Learning rate for our optimizer

# Convenience and saving
ABRIDGED_TRAINING = False # Set to True to train on 10% of the training data, for quick funcitonality tests etc
SAVE_AFTER_TRAINING = True

In [ ]:
# Load data
data = pd.read_csv(DATA_DIR+"train_metadata.csv")
data['filepath'] = AUDIO_DIR + data['filename']

# We only need the filepath and species label
data = data[['filepath', 'primary_label']]

# Replace string labels by tensors whose entries are dummies
species = data['primary_label'].unique()
species_to_index = {species[i]:i for i in range(len(species))}
data['tensor_label'] = pd.Series(pd.get_dummies(data['primary_label']).astype(int).values.tolist()).apply(lambda x: torch.Tensor(x))
data.sample(5)

In [ ]:
# Train test split, stratified by species
data_train, data_test = train_test_split(data, test_size = 0.2, stratify=data['primary_label'])

if ABRIDGED_TRAINING == True:
    data_train = data_train.sample(int(len(data_train)*0.1))

print(data_train.info())

# 2. Preprocessing and Generating Spectrograms

In [ ]:
# Transforms audio signal to a spectrogram
spectrogram_transform = torchaudio.transforms.Spectrogram(
        n_fft=2048,
        win_length=2048,
        hop_length=512,
        power=2
    )

In [ ]:
# Converts ordinary spectrogram to Mel scale
mel_spectrogram_transform = torchaudio.transforms.MelScale(
    n_mels=256,
    sample_rate=SAMPLE_RATE,
    f_min=0,
    f_max=16000,
    n_stft=1025  # the number of frequency bins in the spectrogram
)

In [ ]:
# Scales decibels to reasonable level (apply to a spectrogram or Mel spectrogram)
db_scaler = torchaudio.transforms.AmplitudeToDB(stype="power", top_db=80)

In [ ]:
# Resizes spectrograms into square images
resize = transforms.Resize((224, 224), antialias = None)

In [ ]:
# Takes a filepath and outputs a torch tensor with shape (1, 224, 224) 
# that we can feed into our CNN
def filepath_to_tensor(filepath):
    sample, _ = torchaudio.load(filepath)
    if len(sample) >= SAMPLE_RATE * SAMPLE_LENGTH:
        sample = sample[:SAMPLE_RATE * SAMPLE_LENGTH]
    else:
        pad_length = SAMPLE_RATE * SAMPLE_LENGTH - len(sample)
        sample = torch.nn.functional.pad(sample, (0, pad_length))
    spec = spectrogram_transform(sample)
    mel_spec = mel_spectrogram_transform(spec)
    db_scaled_mel_spec = db_scaler(mel_spec)
    resized = resize(db_scaled_mel_spec)
    return resized

In [ ]:
# Visualize a random spectrogram
t = filepath_to_tensor(data_train.sample()['filepath'].iloc[0])
plt.imshow(t.squeeze().numpy(), cmap='gray')
plt.show()

# 3. Defining Our Torch Dataset Class

In [ ]:
# Note: filepaths and labels should be ordinary lists
class birdCLEF_dataset(Dataset):
    def __init__(self, filepaths, labels):
        super().__init__()
        self.filepaths = filepaths
        self.labels = labels

    def __len__(self):
        return len(self.filepaths)

    def __getitem__(self, index):
        processed_clip = filepath_to_tensor(self.filepaths[index])
        return processed_clip, self.labels[index]

# 4. Defining neural network architecture 

In [ ]:
class birdClassifier(nn.Module):
    ''' Pared down architecture from https://github.com/musikalkemist/pytorchforaudio/blob/main/10%20Predictions%20with%20sound%20classifier/cnn.py'''
    def __init__(self, num_classes):
        super(birdClassifier, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=16,
                out_channels=32,
                kernel_size=5,
                stride=2,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.dropout = nn.Dropout(p=0.1)
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(25088, num_classes)

    def forward(self, input_data):
        x = self.conv1(input_data)
        x = self.conv2(x)
        x = self.dropout(x)
        x = self.flatten(x)
        logits = self.linear(x)
        return logits

# 6. Training

In [ ]:
# Set device we'll train on
device = None
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"Using {device}")

In [ ]:
# Instantiate our dataset
train_dataset = birdCLEF_dataset(filepaths = data_train['filepath'].to_list(), labels = data_train['tensor_label'].to_list())
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# Instantiate our model
model = birdClassifier(NUM_SPECIES).to(device)

In [ ]:
# Set our loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)

In [ ]:
# Training loop
print_per_n_batches = 10
print(f"Training on {len(train_dataset)} samples with {BATCH_SIZE} samples per batch.")

torch.enable_grad() # Turn on the gradient

for epoch in tqdm(range(NUM_EPOCHS)):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(tqdm(train_dataloader, leave = False)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = criterion(outputs, labels)
        loss.backward()

        # Update weights
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % print_per_n_batches == print_per_n_batches-1:    # print after some batches
            print(f'[{epoch + 1}/{NUM_EPOCHS}, {(i + 1):5d}/{len(train_dataloader)}] loss: {running_loss / print_per_n_batches:.3f}')
            running_loss = 0.0

print('Finished Training')

# 6. Save the model

In [ ]:
if SAVE_AFTER_TRAINING == True:
    torch.save(model.state_dict(), str(pd.Timestamp.now())+".pt")

# 7. Validation

In [ ]:
test_dataset =  birdCLEF_dataset(filepaths = data_test['filepath'].to_list(), labels = data_test['tensor_label'].to_list())
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

torch.no_grad() # Turn off gradient for fast validation loss computations
model.eval()

losses = []
for i, data in enumerate(tqdm(test_dataloader)):
    inputs, labels = data
    pred = model(inputs)
    losses += [float(criterion(pred, labels))]

In [ ]:
losses = pd.Series(losses)
losses.index = data_test.index
data_test['loss'] = losses

In [ ]:
print(f"Average loss on test data: {data_test['loss'].sum() / len(data_test['loss'])}")

In [ ]:
average_loss_by_species = pd.DataFrame(data_test.groupby(by = ['primary_label'])['loss'].mean()).sort_values(by='loss', ascending = False)
fig, ax = plt.subplots(figsize=(24, 8))
freqs = sns.barplot(data=average_loss_by_species, x='primary_label', y='loss', ax=ax)
freqs.set_xlabel("")
plt.xticks(rotation=90);

In [ ]:
number_sample_per_species = data_train['primary_label'].value_counts().reindex(average_loss_by_species.index)
number_sample_per_species = pd.DataFrame(number_sample_per_species)
fig, ax = plt.subplots(figsize=(24, 8))
freqs = sns.barplot(data=number_sample_per_species, x='primary_label', y='count', ax=ax)
freqs.set_title("Number of training samples per species")
freqs.set_xlabel("primary_label")
freqs.set_ylabel("Count")
plt.xticks(rotation=90);